In [1]:
import numpy as np
X = []
y = []
with open('train.csv') as f:
    for line in f:
        y.append(int(line[0]))
        X.append(line[5:-6])
y = np.array(y)


In [2]:
import io
def loadDataSet(test_fileName, train_fileName):
    test = []
    train = []
    with io.open(test_fileName, encoding = 'utf-8') as f:
        for line in f:
            lineArr = line.strip().split('\t')
            test.append(lineArr)
    with io.open(train_fileName, encoding = 'utf-8') as f:
        for line in f:
            lineArr = line.strip().split('\t')
            train.append(lineArr)
    return test,train

In [3]:
test, train = loadDataSet('X_test.txt', 'X_train.txt')

In [4]:
def words2Vectors_FP(wordList, data):
    vec = [0] * len(wordList)
    for word in data:
        if word in wordList:
            vec[wordList.index(word)] = 1 # + (word in badList) 
            # count bad words twice
    return vec

def createWordList(X, minchar):
    wordContent = []
    wordCount = dict()
    for line in X:
        for word in line:
            if len(word) > minchar:
                if word in wordCount.keys():
                    wordCount[word] = wordCount[word] + 1 # + (word in badList)
                else:
                    wordCount[word] = 1 # + (word in badList)
            else:
                continue
    return wordCount.keys(), wordCount

In [5]:
wordList, c =  createWordList(train, 2 )
vectors_FP = []
for x in train:
    vectors_FP.append(words2Vectors_FP(wordList, x))
vectors_FP_test = []
for x in test:
    vectors_FP_test.append(words2Vectors_FP(wordList, x))

In [16]:
# on calcule 
def proximal_l1(w, lam,rho):
    result = []
    lam = rho * lam
    for wi in w:
        if wi > lam:
            result.append(wi - lam)
        elif np.abs(wi) <= lam:
            result.append(0)
        else :
            result.append(wi + lam)
    return result

def function_f(w, X, Y, rho):
    X = np.c_[X, np.ones(X.shape[0])] # Add one cols
    sum_f = 0
    k = 0
    for xi, yi in zip(X, y):
        exp = np.exp(-1. * yi * xi.T.dot(w))
        sum_f = sum_f + np.log1p(exp) 
    sum_f = sum_f / y.shape[0]
    return sum_f 

# get value w (30,0) 
def grandient_f(w, X, Y, rho):
    X = np.c_[X, np.ones(X.shape[0])] # Add one cols
    sum_gradient = np.zeros(X.shape[1])
    for xi, yi in zip(X, Y):
        exp = np.exp(-1. * yi * xi.T.dot(w))
        sum_gradient = sum_gradient + xi* (-1.* yi) * (1. - (1. / (1. + exp)))   
    sum_gradient = sum_gradient / y.shape[0]
    return sum_gradient 
# return value of g is (31,) h is (31, 31)

# question 2.2
def optimisation_proximal(X,y,function_f, grandient_f, proximal_l1, rho,x0, rtol, maxloop, lam, beta):
    xk = x0
    fk_old = np.inf
    k = 0

    fk, grad_fk = function_f(xk, X, y, rho), grandient_f(xk,X,y,rho)
    while True :
        k = k + 1
        grad_fk = grandient_f(xk,X,y,rho)
        while True:  #lam change
            xk_grad = xk - lam * grad_fk
            prx = proximal_l1(xk_grad, lam, rho)
            Gt = (xk - prx) / lam
            lhand = function_f(xk - lam * Gt, X,y,rho)
            rhand = fk - lam * grad_fk.dot(Gt) + (0.5 * lam) * Gt.dot(Gt)
            if lhand <= rhand:
                break
            else:
                lam *= beta

        xk -= lam * Gt
        fk_old = fk
        fk, grad_fk = function_f(xk,X,y,rho), grandient_f(xk,X,y,rho)
        if np.linalg.norm(lam * np.array(proximal_l1(xk,lam,rho)) - np.zeros(xk.shape[0])) < rtol or k > maxloop: # stop condition
            print fk
            return xk

In [21]:
class Regression_logstic():
    def __init__(self,C ,rtol = 10**-10, maxloop=1000, lam=0.5, beta = 0.5):
        # initial
        self.rtol = rtol
        self.maxloop = maxloop
        self.lam = lam
        self.beta = beta
        self.rho = 1. / C
        # first col is flag, second col is the evalue
    def get_params(self, deep=True):
        return "c: +" + str(self.C) + str(self.tol)
    def fit(self, X, y):
        self.rho = 1./ y.shape[0]
        self.w = np.zeros(features.shape[1] + 1)
        self.w = optimisation_proximal(X, y, function_f, grandient_f, proximal_l1, self.rho,self.w , self.rtol ,self.maxloop,  self.lam, self.beta)  
    def predict(self, X):
        X = np.c_[X, np.ones(X.shape[0])] # Add one cols
        result = X.dot(self.w)
        result[result >0 ] = 1
        result[result < 0] = 0
        return result.astype('int').tolist()
    
    def score(self, X, y):
        return np.mean(self.predict(X) == y)

In [8]:
y[y == 0] = -1.
from sklearn.feature_selection import chi2,SelectKBest
select = SelectKBest(chi2, 2450)
features = select.fit_transform(vectors_FP, y)
features_test = select.transform(vectors_FP_test)

In [22]:
clf = Regression_logstic(C = 1.)
clf.fit(features, y)
result = clf.predict(features_test)


0.44390122164


In [57]:
print clf.w
# print clf.predict(features_test)
clf_1 = Regression_logstic()
clf_1.w = clf.w
clf_1.predict(features_test)

[ 0.          0.04015366  0.08295721 ...,  0.          0.         -1.25664105]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,


In [6]:
# rho = 1./ y.shape[0]
# w = np.zeros(features.shape[1] + 1)
# w = optimisation_proximal(features, y, function_f, grandient_f, proximal_l1, w , 10*-10 ,1000,  0.5, 0.5)

In [23]:
def predict(w, X):
    X = np.c_[X, np.ones(X.shape[0])] # Add one cols
    result = X.dot(w)
    result[result >0 ] = 1
    result[result < 0] = 0
    return result.astype('int').tolist()

In [ ]:
final = predict(w, features_test)
print final

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_selection import chi2,SelectKBest
clf = SVC(kernel='linear', gamma=2)
select = SelectKBest(chi2)
pipe = Pipeline(steps=[('chi2', select), ('SVM', clf)])

In [40]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', gamma=2)

In [41]:
clf = regression_logstic()


<bound method SVC.get_params of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=2, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)>

0.44390122164


[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [23]:
np.savetxt('y_pred_C_1.txt', result, fmt='%s')

In [6]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_selection import chi2,SelectKBest
clf = SVC(kernel='linear', gamma=2)
select = SelectKBest(chi2)
pipe = Pipeline(steps=[('chi2', select), ('SVM', clf)])
para = {"chi2__k" : np.arange(50, 3100, 200)}
estimator = GridSearchCV(pipe,para)
estimator.fit(vectors_FP, y)
print "result withbigram !"
print estimator.best_score_

/Users/dexter/anaconda/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:165: RuntimeWarning: invalid value encountered in divide
  chisq /= f_exp


result withbigram !
0.796828992072


In [1]:
print vectors_FP[0]

NameError: name 'vectors_FP' is not defined